**Mastodon Toot Scraper** \
This notebook is responsible for scraping all the toots with certain hashtags and cleaning the datasets after. 

In [ ]:
#Import the necessary dependencies 
import platform
import sys
print("This jupyter notebook is running on Python " + platform.python_version())
assert sys.version_info >= (3, 6) # It's good practice to assert packages requirements at the beginning of a script:
from IPython.display import HTML #via the api we get raw html scripts, so  use this function to format it nicely
from mastodon import Mastodon
import csv
from datetime import datetime
from bs4 import BeautifulSoup  # Import BeautifulSoup for HTML parsing
import pandas as pd
import re
import os
import matplotlib.pyplot as plt

In [3]:
!pip install mastodon.py

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
#Creates the API instance
Mastodon.create_app(
    'your_app_name',
    api_base_url = 'https://mastodon.nl', 
    to_file = 'your_client_crediential_file_name.secret'
)


('4RvXItrNVhNRiOAXnfpUdmJddgo9BKm68P8Yg5zDWSo',
 'gYobDSHxzCxOEn8J9OML8GvzGnWmZLQ_5xOuCg_SxL8')

In [5]:
#Performs the login of the API instance 
API = Mastodon(client_id = 'your_client_crediential_file_name.secret',)
API.log_in(
    'rickvanderluijt@hotmail.com', #no caps
    '@Social123',
    to_file = 'your_user_crediential_file_name.secret'
)

'DygfO7tnROAcQGxMKZX1AbkhrXFIx5-c9r-cRI-0itY'

In [6]:
#View instances of the API and limitations 
API.instance()

{'uri': 'mastodon.nl',
 'title': 'Mastodon.nl door Stichting Activityclub',
 'short_description': 'Mastodon voor de Nederlandse community - door de Nederlandse community - in stand gehouden door Stichting Activityclub',
 'description': '<p>Mastodon-instance voor de <em>Nederlandse gemeenschap</em>, maar <em>open voor iedereen</em>. Voertaal voornamelijk Nederlands en Engels.</p>\r\n\r\n<hr style="border-style: dashed;" />\r\n\r\n<p>Mastodon instance for the <em>Dutch community</em>, but <em>open to anyone</em>. Primary languages Dutch and English.</p>\r\n\r\n<p>Maintained by <a href="mailto:mdbraber@mastodon.nl">mdbraber@mastodon.nl</a> and <a href="mailto:eelco@maljaars.net">eelco@maljaars.net</a>.\r\n',
 'email': 'team@mastodon.nl',
 'version': '4.2.13',
 'urls': {'streaming_api': 'wss://mastodon.nl'},
 'stats': {'user_count': 41800,
  'status_count': 2544032,
  'domain_count': 41728},
 'thumbnail': 'https://mastodon.nl/system/site_uploads/files/000/000/003/@1x/993d37c159224462.png',

In [7]:
#See what statusses are trending inside the server to just check if its performing correctly 
from IPython.display import HTML #for formatting HTML script
HTML(API.trending_statuses()[2]["content"])

In [8]:
#Get the 3 most popular tags inside the server for same reasons as before
tags = API.trending_tags(limit=3)  # Get the list of tags
# print(tags)
names = [tag['name'] for tag in tags]  # Extract the 'name' from each dictionary
print(names)


['meermittwoch', 'musicwomenwednesday', 'spotifywrapped']


In [9]:
#Print the 10 most recent toots with the specified 'q' hashtag 
q = "#politiek"
search_results = API.search_v2(q)
for n in range(10):
    print(f"Toot {n+1}\n\n")
    html = search_results["statuses"][n]["content"]
    display(HTML(html))
    print("_"*100)

Toot 1




____________________________________________________________________________________________________
Toot 2




____________________________________________________________________________________________________
Toot 3




____________________________________________________________________________________________________
Toot 4




____________________________________________________________________________________________________
Toot 5




...

____________________________________________________________________________________________________
Toot 7




____________________________________________________________________________________________________
Toot 8




____________________________________________________________________________________________________
Toot 9




____________________________________________________________________________________________________
Toot 10




____________________________________________________________________________________________________


In [10]:
# Gathers the last n toots based on specified query hashtag
# Query and parameters
q = "#politiek"
max_toots = 10  # Total number of toots to retrieve
toots_retrieved = 0
max_id = None  # Used for pagination

# Fetch toots with pagination
while toots_retrieved < max_toots:
    # Perform the search
    search_results = API.search_v2(q, max_id=max_id)
    
    # Check if the API returned any results
    statuses = search_results.get("statuses", [])
    if not statuses:
        break  # No more results
    
    # Process each toot
    for status in statuses:
        toots_retrieved += 1
        
        # Display the toot and date/time
        print(f"Toot {toots_retrieved}\n")
        html = status["content"]
        display(HTML(html))
        print(f"Date/Time: {status['created_at']}")
        print("_" * 100)
        
        # Stop if we've retrieved the desired number of toots
        if toots_retrieved >= max_toots:
            break
    
    # Update max_id to fetch older toots
    max_id = statuses[-1]["id"]


Toot 1



Date/Time: 2024-12-04 10:05:57.482000+00:00
____________________________________________________________________________________________________
Toot 2



Date/Time: 2024-12-04 08:56:36+00:00
____________________________________________________________________________________________________
Toot 3



Date/Time: 2024-12-04 07:44:19.535000+00:00
____________________________________________________________________________________________________
Toot 4



Date/Time: 2024-12-04 05:23:29.678000+00:00
____________________________________________________________________________________________________
Toot 5



...

Date/Time: 2024-10-25 16:32:04+00:00
____________________________________________________________________________________________________
Toot 298



Date/Time: 2024-10-25 15:49:47+00:00
____________________________________________________________________________________________________
Toot 299



Date/Time: 2024-10-25 14:31:13+00:00
____________________________________________________________________________________________________
Toot 300



Date/Time: 2024-10-25 14:17:47+00:00
____________________________________________________________________________________________________


KeyboardInterrupt: 

In [20]:
# Gathers the last n toots based on specified query hashtag
# Then writes these toots into a .csv based on the hashtag
# Query and parameters
q = "#kabinet "
max_toots = 1000  # Total number of toots to retrieve
toots_retrieved = 0
max_id = None  # Used for pagination

# Open CSV file for writing
with open('kabinet.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Username', 'Content', 'Date/Time'])  # Writing header

    # Fetch toots with pagination
    while toots_retrieved < max_toots:
        # Perform the search
        search_results = API.search_v2(q, max_id=max_id)
        
        # Check if the API returned any results
        statuses = search_results.get("statuses", [])
        if not statuses:
            break  # No more results
        
        # Process each toot
        for status in statuses:
            toots_retrieved += 1
            username = status["account"]["username"]
            content = status["content"]
            date_time = status["created_at"]
            
            # Format the date/time
            date_time_formatted = date_time.strftime("%Y-%m-%d %H:%M:%S")
            
            # Remove HTML tags from content
            clean_content = re.sub(r'<.*?>', '', content)  # Using regex to remove HTML tags
            
            # Write the data to the CSV file
            writer.writerow([username, clean_content, date_time_formatted])
        
        # Update max_id to fetch older toots
        max_id = statuses[-1]["id"]

KeyboardInterrupt: 

In [18]:
df = pd.read_csv('kabinet.csv')
df.tail()

,Username,Content,Date/Time
553,redegelde,Goedemorgen #conciërges #EduTooters ook deze z...,2022-11-07 05:45:11
554,redegelde,#onderwijs ik prijs me gelukkig met huidige le...,2022-11-06 18:50:06
555,redegelde,nog twijfelende #onderwijs #tweeps? https://ma...,2022-11-05 09:36:32
556,redegelde,@sschouwenaars ja hoor. Ik heb geen blokkade b...,2022-11-05 07:19:35
557,wimpelgrim,Ik maak zelf sinds 2019 een onderwijspodcast e...,2019-10-25 08:00:42


In [22]:
# Checks Rate limit for server remaning not essential to code but nice to know as API can give some constraints.
API.ratelimit_remaining

0

In [ ]:
# Checks and drops duplicates
# df = df.drop_duplicates('column_name', keep='last')

In [23]:
#DROPPING DUPLCIATES in all csv files in related folder location

# Step 1: Remove duplicates from each CSV file

folder_path = './Kopie_csv/'  # Adjust to your folder path containing the CSV files
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]  # List all CSV files in the folder

for file in all_files:
    file_path = os.path.join(folder_path, file)
    
    # Read the CSV into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Remove duplicates based on the "Content" column
    df_cleaned = df.drop_duplicates(subset=["Content"])
    
    # Save the cleaned DataFrame back to a new CSV file
    cleaned_file_path = os.path.join(folder_path, f"cleaned_{file}")
    df_cleaned.to_csv(cleaned_file_path, index=False)

    print(f"Removed duplicates from {file}. Saved cleaned version as cleaned_{file}.")

Removed duplicates from gezondheidszorg.csv. Saved cleaned version as cleaned_gezondheidszorg.csv.
Removed duplicates from immigratie.csv. Saved cleaned version as cleaned_immigratie.csv.
Removed duplicates from kabinet.csv. Saved cleaned version as cleaned_kabinet.csv.
Removed duplicates from klimaat1.csv. Saved cleaned version as cleaned_klimaat1.csv.
Removed duplicates from onderwijs.csv. Saved cleaned version as cleaned_onderwijs.csv.
Removed duplicates from politiek1.csv. Saved cleaned version as cleaned_politiek1.csv.


In [25]:
folder_path = './Kopie_csv/'  # Adjust to your folder path containing the cleaned CSV files
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f.startswith('cleaned_')]  # List all cleaned CSV files

# Create an empty list to store DataFrames
all_toos_df_list = []

for file in all_files:
    cleaned_file_path = os.path.join(folder_path, file)
    if os.path.exists(cleaned_file_path):  # Check if cleaned file exists
        # Read the cleaned CSV into a DataFrame
        temp_df = pd.read_csv(cleaned_file_path)
        
        # Append the DataFrame to the list
        all_toos_df_list.append(temp_df)

# Concatenate all DataFrames in the list into a single DataFrame
all_toos_df = pd.concat(all_toos_df_list, ignore_index=True)

# Save the combined DataFrame to a new CSV file
all_toos_df.to_csv(os.path.join(folder_path, "All_toots.csv"), index=False)

print("All cleaned toots have been combined into All_toots.csv.")

All cleaned toots have been combined into All_toots.csv.


In [26]:
# Load the All_toots CSV file into a DataFrame
all_toos_df = pd.read_csv('./Kopie_csv/All_toots.csv')  # Adjust the path if necessary

# Check for duplicates based on the 'Content' column
duplicates = all_toos_df[all_toos_df.duplicated(subset=['Content'], keep=False)]

# Display duplicates if any
if not duplicates.empty:
    print(f"Found {len(duplicates)} duplicate rows based on 'Content'.")
else:
    print("No duplicates found based on 'Content'.")

# Remove duplicates (keeping the first occurrence)
all_toos_df_cleaned = all_toos_df.drop_duplicates(subset=['Content'], keep='first')

# Save the cleaned DataFrame back to CSV
all_toos_df_cleaned.to_csv('All_toots_cleaned.csv', index=False)

print("Duplicates removed and saved to All_toots_cleaned.csv.")

Found 212 duplicate rows based on 'Content'.
Duplicates removed and saved to All_toots_cleaned.csv.
